In [14]:
%load_ext autoreload
%autoreload 2

from dynamic_system_functions import *
import casadi as ca

s = ca.SX.sym('s')

M = ca.SX.sym('M')
B = ca.SX.sym('B')
Kp = ca.SX.sym('Kp')
Kd = ca.SX.sym('Kd')

R = sys([Kd,Kp,0],[M,B,Kp])

D = sys([1],[1])#pade(8.0e-4, 3)

Kl = ca.SX.sym('Kl')
Cff = sys([Kl, 1], [1])

B_real = ca.SX.sym('B_real')
a_vel = ca.SX.sym('a_vel')
P_hat = ca.SX.sym('P_hat')
a_acc = ca.SX.sym('a_acc')
Cfb_Ov = sys([B_real*a_vel, 0],[1, a_vel, 0])
P_hat_Oa = sys([-P_hat*a_acc*a_vel, 0, 0],[1, a_acc+a_vel, a_acc*a_vel, 0])

Ma = ca.SX.sym('Ma')
Ba = ca.SX.sym('Ba')
A = sys([1], [Ma, Ba, 0])


p = [M, B, Kp, Kd] # fixed params
x = [Kl, Ma, Ba, B_real, a_vel, P_hat, a_acc]   # dec variables
vars = [*p, *x]    # list of all the variables, useful for making casadi functions

##### Building some SYSTEMS ######
# OL from F to v
G = R*D*Cff*A
D_fb = sys([1],[1]) + R*Cfb_Ov# +G*P_hat_Oa

# Environment contact denominator
K_env = ca.SX.sym('K_env')
M_env = ca.SX.sym('M_env')
E = sys([M_env, 0, K_env], [1, 0])

G_cl = fb(G, E)
G_cl_den_fn = ca.Function('G_cl_den_fn', [K_env, M_env, *vars], G_cl.den)

# Noise TFs
# OL noise
noise_area_of_concern = sys([20, 0], [1, 20])
G_nf = D*Cff*A*noise_area_of_concern
# CL noise

#G_nf_num = con(con(con(con(D_num, Cff_num), A_num), E_den), G_den)
#G_nf_den = con(con(con(G_cl_den, D_den), Cff_num), A_num)

nf_norm_fn = ca.Function('nf_norm_fn', [K_env, M_env, *vars], [G_nf.h2_norm()])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Cancelling pole/zero at 0
@1=20, 
[[0, 1], 
 [(-((@1*Ba)/Ma)), (-(((@1*Ma)+Ba)/Ma))]]


In [18]:
print(D_fb)

num: [SX(M), SX((((M*a_vel)+B)+(Kd*(B_real*a_vel)))), SX((((B*a_vel)+Kp)+(Kp*(B_real*a_vel)))), SX((Kp*a_vel)), SX(0)] den: [SX(M), SX(((M*a_vel)+B)), SX(((B*a_vel)+Kp)), SX((Kp*a_vel)), SX(0)]


In [22]:
# Formulate optimization objective:
# maximize admittance 
# s.t.:
p0 =  [5, 120, 400, 100] # numerical value of params
x0 =  [0.01, 12,  500,  0.5, 10, 12, 10]   # initial guess at dec vars
ubx = [0.06, 30, 3000, 10.0,  60, 40, 60]  # upper bound
lbx = [0.0,   5,  100,  0.0, 0.0, 0.0, 0.0]# lower bound
noise_bound = 3e-3 # 1e-3 is more realistic, but let's see
M_en = 12.0        # Env mass in Kg

# OBJECTIVE: J will be minimized, to maximize admittance F->v
#Am, Bm, Cm = tf2ss(G_num, G_den)
J = -(G/D_fb).h2_norm() #H2_norm(Am, Bm, Cm)

# Scale the objective
J_fn = ca.Function('J_fn', [*vars], [J])
J_grad_fn = ca.Function('J_grad_fn', [*vars], [ca.gradient(J, ca.horzcat(*x))])
J0 = J_fn(*p0, *x0)
print('Objective: {}'.format(J0))
print('Gradient:  {}'.format(J_grad_fn(*p0, *x0)))
print('Noise fn:  {}'.format(nf_norm_fn(1e5, M_en, *p0, *x0)))
J *= 1/ca.fabs(J0)

# CONSTRAINTS
g = []
ubg = []
lbg = []

# OL Stability:
rt = instab(G.den)
for r_i in range(1, len(rt)):
    g.append(rt[r_i]*rt[0])
    ubg.append(np.Inf)
    lbg.append(1e-8) 
    
# Stability and noise in contact:
for K_en in [3e5]:
    #Stab
    G_cl_den = G_cl_den_fn(K_en, M_en, *vars)
    rt = instab(G_cl_den)
    for r_i in range(1, len(rt)):
        g.append(rt[r_i]*rt[0])
        ubg.append(np.Inf)
        lbg.append(1e-8)    

    #Noise
    g.append(nf_norm_fn(K_en, M_en, *vars))
    ubg.append(noise_bound)
    lbg.append(-np.Inf)

    
opts = { 
         'ipopt.mu_strategy': 'monotone',        # Determines which barrier parameter update strategy is to be used. Default "monotone", can choose 'adaptive' instead
         'ipopt.mu_init' : 0.001,                # This option determines the initial value for the barrier parameter (mu). It is only relevant in the monotone, Fiacco-McCormick version of the algorithm.
       }
        
# J is objective, w decision vars, g constraints (subject to ubg and lbg)
prob = dict(f = J, x = ca.vertcat(*x), g = ca.vertcat(*g), p = ca.vertcat(*p))
solver = ca.nlpsol('solver', 'ipopt', prob, opts)
args = dict(x0 = x0, lbx = lbx, ubx = ubx, ubg = ubg, lbg = lbg, p = p0)
sol = solver(**args)
print(x0)
print(sol['x'].full())

Cancelling pole/zero at 0
Cancelling pole/zero at 0
Objective: -0.00777046
Gradient:  [[-0.0169199, 0.000454444, 4.63426e-06, 0.00635082, -8.13298e-05, 0, 0]]
Noise fn:  0.00541103
This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        8
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        7
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        7
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        2
   inequality constraints with lowe

CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:30 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).")

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  10 -3.6428842e+00 2.29e-03 1.04e+07  -3.0 3.14e+01   0.4 1.00e+00 4.45e-03f  1
  11 -3.9641843e+00 2.23e-03 2.54e+07  -3.0 2.35e+02  -0.1 1.00e+00 2.51e-02h  5
  12 -4.4475448e+00 2.23e-03 1.04e+07  -3.0 2.79e+02  -0.5 1.00e+00 1.86e-09h 30
  13 -1.2339818e+01 1.31e-03 6.11e+06  -3.0 2.80e+02  -1.0 3.72e-01 5.00e-01f  2
  14 -5.0555612e+00 1.15e-03 4.09e+05  -3.0 2.39e+02  -1.5 1.00e+00 1.25e-01h  4
  15 -1.6305731e+01 1.10e-03 1.36e+06  -3.0 6.41e+02    -  1.00e+00 4.73e-02h  3
  16 -7.0237304e+00 8.48e-04 1.63e+06  -3.0 1.99e+02  -1.1 9.88e-01 2.45e-01h  1
  17 -1.1075530e+01 7.34e-04 6.48e+05  -3.0 1.71e+02  -0.6 2.54e-03 1.40e-01h  3
  18 -2.3826352e+01 7.32e-04 2.80e+08  -3.0 5.88e+00   3.4 1.00e+00 1.53e-03h  4
  19 -1.4257835e+01 7.24e-04 2.31e+09  -3.0 6.04e+00   3.8 1.00e+00 1.46e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  20 -1.3375044e+01 7.16e-04

2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:31 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../cas

  21 -3.6094799e+01 3.91e-04 1.11e+10  -3.0 1.87e+02    -  8.01e-01 5.00e-01h  2
  22 -2.4517891e+01 3.84e-04 2.23e+10  -3.0 1.07e+02   2.9 1.00e+00 1.78e-02h  1
  23 -1.0019239e+02 2.59e-04 3.87e+09  -3.0 1.22e+02    -  8.13e-01 3.39e-01f  2
  24 -4.4556042e+01 7.87e-05 1.34e+09  -3.0 7.60e+01   2.4 1.00e+00 7.52e-01h  1
  25 -5.7988039e+01 6.79e-05 3.43e+09  -3.0 5.27e+01    -  9.30e-01 1.37e-01f  2
  26 -1.2706376e+02 3.23e-05 1.82e+09  -3.0 4.63e+01    -  1.00e+00 5.31e-01f  1
  27 -4.3139343e+01 3.23e-05 1.95e+09  -3.0 2.44e+01    -  1.00e+00 3.65e-05h  4
  28 -9.4402128e+01 3.54e-07 4.45e+04  -3.0 2.29e+01    -  7.88e-01 1.00e+00h  1
  29 -5.1361882e+01 3.53e-07 3.87e+07  -3.0 4.46e+00    -  9.98e-01 9.65e-04h  3
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  30 -1.2308012e+02 4.32e-07 3.05e+07  -3.0 5.86e+02    -  5.47e-01 2.13e-01f  1
  31 -1.4546439e+02 4.31e-07 3.05e+07  -3.0 1.99e+02    -  1.00e+00 1.10e-03h  2
  32 -9.8324459e+01 3.76e-07

...

  86r-1.0612936e+02 0.00e+00 4.92e-06  -9.0 3.69e-02    -  1.00e+00 1.00e+00h  1
  87r-1.2275839e+02 0.00e+00 1.29e-03  -9.0 7.57e-01    -  1.00e+00 1.95e-03h 10
  88r-9.7679706e+01 0.00e+00 1.48e-03  -9.0 8.72e-01    -  1.00e+00 9.77e-04h 11


2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../cas

  89r-1.5049134e+02 0.00e+00 1.08e-06  -9.0 8.96e-01    -  1.00e+00 1.00e+00H  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  90r-7.8389514e+01 0.00e+00 4.84e-05  -9.0 5.63e-02    -  1.00e+00 5.00e-01h  2
  91r-3.0651256e+01 0.00e+00 5.37e-09  -9.0 2.52e-02    -  1.00e+00 1.00e+00H  1
  92r-9.0351737e+01 0.00e+00 4.46e-04 -11.0 6.21e-01    -  1.00e+00 5.00e-01h  2
  93r-1.3911537e+02 0.00e+00 1.38e-07 -11.0 1.15e-01    -  1.00e+00 1.00e+00h  1
  94r-1.0666363e+02 0.00e+00 2.61e-05 -11.0 3.08e-01    -  1.00e+00 5.00e-01h  2
  95r-8.7824192e+01 0.00e+00 3.63e-05 -11.0 2.21e-01    -  1.00e+00 3.12e-02h  6
  96r-7.7613234e+01 0.00e+00 3.72e-05 -11.0 2.26e-01    -  1.00e+00 3.12e-02h  6
  97r-1.2195204e+02 0.00e+00 2.82e-05 -11.0 2.21e-01    -  1.00e+00 2.50e-01h  3
  98r-2.5368397e+01 0.00e+00 2.51e-05 -11.0 1.69e-01    -  1.00e+00 1.25e-01h  4
  99r-5.1969558e+01 0.00e+00 2.49e-05 -11.0 1.49e-01    -  1.00e+00 1.56e-02h  7


CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).") [.../casadi/core/oracle_function.cpp:265]
CasADi - 2021-07-02 17:59:34 WARNING("solver:nlp_f failed: NaN detected for output f, at (row 0, col 0).")

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 100r-1.0548998e+02 0.00e+00 2.48e-05 -11.0 1.47e-01    -  1.00e+00 1.95e-03h 10
 101r-4.7722887e+01 0.00e+00 2.48e-05 -11.0 1.46e-01    -  1.00e+00 9.77e-04h 11
 102r-7.9318472e+01 0.00e+00 2.48e-05 -11.0 1.46e-01    -  1.00e+00 1.95e-03h 10
 103r-1.1435762e+02 0.00e+00 5.28e-09 -11.0 1.46e-01    -  1.00e+00 1.00e+00H  1
 104r-1.1137543e+02 0.00e+00 6.54e-07 -11.0 3.96e-03    -  1.00e+00 3.12e-02h  6
Restoration phase converged to a feasible point that is
unacceptable to the filter for the original problem.
Restoration phase in the restoration phase failed.

Number of Iterations....: 104

                                   (scaled)                 (unscaled)
Objective...............:  -8.4259436260224291e+01   -8.4259436260224291e+01
Dual infeasibility......:   9.1016963350472417e+02    9.1016963350472417e+02
Constraint violation....:   0.0000000000000000e+00    0.0000000000000000e+00
Complementarity.....

In [ ]:
# Find extrema of imaginary component over frequency
imag_der = der(imag_coeff)
print(imag_coeff)
poly_coeffs = ca.SX.sym('poly',3,1)
poly_coeffs[0] = imag_der[0]
poly_coeffs[1] = imag_der[2]
poly_coeffs[2] = imag_der[4]
roots = ca.sqrt(-ca.poly_roots(poly_coeffs))
print(roots[0])

for j in range(roots.shape[0]):
    tot = 0.0
    num_coeffs = len(imag_coeff)
    for i in range(num_coeffs):
        tot += ca.constpow(roots[j],num_coeffs-i-1)*imag_coeff[i]
    print(tot)
crit_points_imag = []
crit_points_real = []
crit_points_freq = []
for i in range(roots.shape[0]):
    crit_points_freq.append(ca.Function('crit_freq', [M, B, Kp, Kd, Ma, Ba, Kl], [roots[i]]))
    crit_points_imag.append(ca.Function('crit_imag', [M, B, Kp, Kd, Ma, Ba, Kl], [imag_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))
    crit_points_real.append(ca.Function('crit_real', [M, B, Kp, Kd, Ma, Ba, Kl], [real_fn(roots[i], M, B, Kp, Kd, Ma, Ba, Kl)]))

# Verifying the real/imaginary components are realistic
# Can compare with addmittance_check_casadi.m
import numpy as np
import matplotlib.pyplot as plt
Mt = 5
Bt = 10
Kpt = 1
Kdt = 2
Mat = 3
Bat = 5
Klt = 0.1

plt.figure()
plt.clf()
for om in [0.001, 10, 500]:
    print("freq {} real {} im {}".format(om, real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))

for om in np.logspace(-10,10, num = 1000):
    plt.plot(real_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), imag_fn(om, Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'ko')
    
for cp_r, cp_i, cp_f in zip(crit_points_real, crit_points_imag, crit_points_freq):
    print('crit point re: {} im: {} fr: {}'.format(\
                        cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), 
                        cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),
                        cp_f(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt)))
    plt.plot(cp_r(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt), cp_i(Mt, Bt, Kpt, Kdt, Mat, Bat, Klt),'dr')
plt.show()

In [ ]:
# TEST FUNCTIONS
# Test instap
#RT_col = instab([1, 2, 3, 4, 5])
#print(RT_col)

# Test lyap
A = np.array([[-0.1, 0, 1],[0, 1, 0], [0, 0, 1]])
Q = np.array([[1, 0, 0],[0, 1, 0], [0, 3, 1]])
X = lyap(A,Q)
print(X)

# Test H2
A = np.array([[-0.5, 1],[-0.2, -0.1]])
B = np.array([[0],[.3]])
C = np.array([[3, 0]])
print(H2_norm(A, B, C))

# Test tf2ss
num = [1, 3]
den = [1, 5, 10, 20, 20]
A, B, C = tf2ss(num, den)
print(H2_norm(A, B, C))

# Test symbolic diff
b = ca.MX.sym('b')
k = ca.MX.sym('k')
A = ca.MX.zeros(2,2)
A[0,1] = ca.MX(1.0)
A[1,0] = -b
A[1,1] = -k
#Q = ca.MX.eye(2)
norm = H2_norm(A, B, C)
norm_fn = ca.Function('norm', [b, k], [norm])
b_grad_fn = ca.Function('b_grad',[b, k], [ca.gradient(norm,b)])
k_grad_fn = ca.Function('k_grad',[b, k], [ca.gradient(norm,k)])

b0 = 2
k0 = 1.0
h = 0.1
h20 = norm_fn(b0, k0)
bgrad = b_grad_fn(b0, k0)
kgrad = k_grad_fn(b0, k0)
h21 = norm_fn(b0, k0+h*bgrad+h*kgrad)


print('H2 init: {}'.format(h20))
print('Grads:   {}, {}'.format(bgrad, kgrad))
print('H2 fin:  {}'.format(h21))
print('err:     {}'.format(h20+h*bgrad+h*kgrad-h21))

In [ ]:
# OLD: using casadi built-ins
# Build the functions for real/imaginary components of TF
from copy import deepcopy
num = R_num*D_num*Cff_num*A_num
den = R_den*D_den*Cff_den*A_den
# Building the complex conjugate
den_cc_coeff = ca.poly_coeff(den, s)
for i in range(1, den_cc_coeff.shape[0]):
    if i % 2 == 1: # odd power of 's' 
        den_cc_coeff[-i-1] = -den_cc_coeff[-i-1]
den_cc_poly = ca.polyval(den_cc_coeff, s)

# multiply num and den by complex conjugate
num_cc = num*den_cc_poly
den_cc = den*den_cc_poly

# Substitute s = j\omega (turn those j^2, j^4,... into -1s)
num_coeff = ca.poly_coeff(num_cc, s)
den_coeff = ca.poly_coeff(den_cc, s)

for i in range(0, num_coeff.shape[0]): # start at 1 to skip the 0th power of j\omega
    number_of_i_squared = ca.floor(i/2)
    if number_of_i_squared % 2 == 1:   # odd number of i^2 -> -1
        num_coeff[-i-1] *= -1.0
for i in range(0, den_coeff.shape[0]):
    num_i_squared = ca.floor(i/2)
    if num_i_squared % 2 == 1: # odd number of i^2 -> -1
        den_coeff[-i-1] *= -1.0

imag_coeff = deepcopy(num_coeff)
real_coeff = deepcopy(num_coeff)
for i in range(num_coeff.shape[0]):
    if i % 2 == 0: # even power of 's', 
        imag_coeff[-i-1] = 0
    else:
        real_coeff[-i-1] = 0
imag_poly = ca.polyval(imag_coeff,s)
real_poly = ca.polyval(real_coeff,s)
den_poly = ca.polyval(den_coeff,s)
imag_fn = ca.Function('imag_fn', [s, M, B, Kp, Kd, Ma, Ba, Ka], [imag_poly/den_poly])
real_fn = ca.Function('real_fn', [s, M, B, Kp, Kd, Ma, Ba, Ka], [real_poly/den_poly])
